In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)
tf.config.list_physical_devices('GPU')

2.5.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [4]:
path = '/gdrive/My Drive/'

<h1> Load Module </h1>

In [5]:
from keras import models, layers, optimizers, utils
from keras.layers import Dropout, Activation, Dense, BatchNormalization
from keras.models import load_model

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import load_img
import random
import cv2, os, sys
from keras.preprocessing import image
from PIL import Image

In [8]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from numpy import expand_dims

<h1> Load Image </h1>

In [9]:
# 이미지 크기 지정

IMAGE_WIDTH = 192
IMAGE_HEIGHT = 144

In [10]:
# 트레이닝 데이터 파일을 정렬
import natsort
train_order = os.listdir(path+'03_iris_training')
train_order = natsort.natsorted(train_order)

In [11]:
# Image Augmentation 생성
datagen = ImageDataGenerator(
            width_shift_range = 0.01,
            height_shift_range = 0.01,
            horizontal_flip = True,
            vertical_flip = True,
            zoom_range = 0.05,
            brightness_range=[1.0, 1.0])

In [12]:
# 이미지를 64개의 카테고리로 분류
nb_classes = 64

data = []
label= []

for filename in train_order:
    category = filename[:3]
    imagePath = path + '/03_iris_training/' + filename
    
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
    
    data.append(image)
    label.append(int(category)-1)
    new_image = img_to_array(image)
    samples = expand_dims(new_image, 0)
    it= datagen.flow(samples, batch_size=1)

    # 이미지를 10개씩 augmentation
    for i in range(10):
        batch = it.next()
        image = batch[0].astype('uint8')
        np_image = np.array(image)
        data.append(np_image)

        label.append(int(category)-1)

In [13]:
# 훈련데이터셋 형성
train_data = [data, label]
(X_data, Y_data) = (train_data[0], train_data[1])

In [14]:
X_data = np.array(X_data)
Y_data = np.array(Y_data)

In [15]:
print(X_data.shape)
print(Y_data.shape)

(1408, 144, 192, 3)
(1408,)


<h1> Build Model </h1>

In [16]:
from keras import utils as np_utils
from keras.layers import Input, Dense
from keras.applications.resnet_v2 import ResNet50V2

from keras.layers import GlobalAveragePooling2D, BatchNormalization
from keras.metrics import Precision, Recall
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

In [18]:
# ResNet Pre-trained Model 사용

def build_model():

    input = Input(shape = (IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    base_model = ResNet50V2(include_top = False, weights = 'imagenet', input_tensor = input)

    for layer in base_model.layers[:44]:
      layer.trainable = False

    X = base_model.output
    X = BatchNormalization()(X)
    X = GlobalAveragePooling2D()(X)
    X = Dense(512, activation = 'relu')(X)
    X = Dropout(0.5)(X)

    Y = Dense(64, activation='softmax')(X)

    model = tf.keras.Model(inputs = input, outputs = Y)

    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0001), metrics=['accuracy',Precision(),Recall()])
    model.summary()

    return model

In [19]:
model = build_model()
model.summary()

94683136/94668760 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 24,654,912
Trainable params: 24,346,432
Non-trainable params: 308,480
__________________________________________________________________________________________________
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 24,654,912
Trainable params: 24,346,432
Non-trainable params: 308,480
__________________________________________________________________________________________________


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [22]:
# 이미지 훈련을 3회로 분할해 교차검증 진행
skf = StratifiedKFold(n_splits=3)

In [23]:
#모델을 사용한 훈련 진행

all_history = []
all_acc = []
all_f1 = []
all_recall = []
all_precision = []

for train_index, test_index in skf.split(X_data, Y_data):
    X_train, X_test, Y_train, Y_test = train_test_split(X_data,
                                                         Y_data,
                                                         test_size = 0.3,
                                                         random_state=10)
    

    X_train = np.array(X_train)
    X_test = np.array(X_test)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    Y_train = to_categorical(Y_train, nb_classes)
    Y_test = to_categorical(Y_test, nb_classes)

    X_train /= 255
    X_test /= 255

    history = model.fit(X_train, Y_train, epochs = 60, batch_size = 128, verbose = 1)
    all_history.append(history.history)

    Y_pred = model.predict(X_test)
    Y_pred_binary = np.around(Y_pred)

    acc = accuracy_score(Y_test, Y_pred_binary)
    f1 = f1_score(Y_test, Y_pred_binary, average = 'micro')
    recall = recall_score(Y_test, Y_pred_binary, average = 'micro')
    precision = precision_score(Y_test, Y_pred_binary, average = 'micro')
    
    all_acc.append(acc)
    all_f1.append(f1)
    all_recall.append(recall)
    all_precision.append(precision)

Epoch 1/60
8/8 [==============================] - 13s 2s/step - loss: 4.1819 - accuracy: 0.0234 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/60
8/8 [==============================] - 8s 1s/step - loss: 3.8836 - accuracy: 0.0690 - precision: 0.0156 - recall: 5.9581e-05
Epoch 3/60
8/8 [==============================] - 8s 1s/step - loss: 3.4132 - accuracy: 0.1543 - precision: 0.4007 - recall: 0.0020
Epoch 4/60
8/8 [==============================] - 8s 1s/step - loss: 2.7545 - accuracy: 0.2975 - precision: 0.6589 - recall: 0.0111
Epoch 5/60
8/8 [==============================] - 8s 1s/step - loss: 1.6853 - accuracy: 0.5563 - precision: 0.7765 - recall: 0.0404
Epoch 6/60
8/8 [==============================] - 8s 1s/step - loss: 0.5860 - accuracy: 0.8284 - precision: 0.8907 - recall: 0.1234
Epoch 7/60
8/8 [==============================] - 8s 1s/step - loss: 0.4323 - accuracy: 0.8569 - precision: 0.8992 - recall: 0.2206
Epoch 8/60
8/8 [==============================] - 8s 1s/step - 

In [25]:
print('Average accuracy : ', sum(all_acc)/3)
print('Average recall : ', sum(all_recall)/3)
print('Average F1 score : ', sum(all_f1)/3)
print('Average precision : ', sum(all_precision)/3)

Average accuracy :  0.9929078014184397
Average recall :  0.9929078014184397
Average F1 score :  0.9929078014184397
Average precision :  0.9929078014184397


In [26]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.012952746823430061
Test accuracy: 0.9929078221321106


<h1> Test </h1>

In [27]:
import pandas as pd

In [43]:
# 테스트데이터 파일을 숫자 순으로 정렬
import natsort
order_list = os.listdir(path+'03_iris_test')
order_list = natsort.natsorted(order_list)

In [45]:
print(order_list)

['1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png', '31.png', '32.png', '33.png', '34.png', '35.png', '36.png', '37.png', '38.png', '39.png', '40.png', '41.png', '42.png', '43.png', '44.png', '45.png', '46.png', '47.png', '48.png', '49.png', '50.png', '51.png', '52.png', '53.png', '54.png', '55.png', '56.png', '57.png', '58.png', '59.png', '60.png', '61.png', '62.png', '63.png', '64.png', '65.png', '66.png', '67.png', '68.png', '69.png', '70.png', '71.png', '72.png', '73.png', '74.png', '75.png', '76.png', '77.png', '78.png', '79.png', '80.png', '81.png', '82.png', '83.png', '84.png', '85.png', '86.png', '87.png', '88.png', '89.png', '90.png', '91.png', '92.png', '93.png', '94.png', '95.png', '96.png', '97.png', '98.png', '99.png', '100.png', '101.pn

In [44]:
test_dataset = []
tmp = []

for filename in order_list:
    imagePath = path + '/03_iris_test/' + filename
    
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
   
    tmp.append(filename)
    test_dataset.append(image)

In [46]:
test_dataset = np.array(test_dataset)
test_dataset = test_dataset.astype('float32')
test_dataset /= 255
predict = model.predict(test_dataset).argmax(1)

<h1> Result </h1>

In [47]:
result = pd.DataFrame({'Image':[],
                 'Answer':[]})

In [48]:
for i in range(len(predict)):
  new_data = {'Image':int(i+1), 'Answer':int(predict[i])+1}
  result = result.append(new_data, ignore_index = True)

In [49]:
result

,Image,Answer
0,1.0,46.0
1,2.0,59.0
2,3.0,26.0
3,4.0,46.0
4,5.0,51.0
...,...,...
251,252.0,21.0
252,253.0,14.0
253,254.0,26.0
254,255.0,45.0


In [50]:
result.to_csv('/gdrive/My Drive/iris_result.csv')